In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

In [4]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

,Unnamed: 0,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,...,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop
1780,1780,Coos,New Hampshire,NH,27311,29128,Coos,"POLYGON ((-71.764977 44.406586999999995, -71.7...",33007.0,5.8,...,18.1,1.55,147726,124677,136816,165164,129090,120889,0.429892,0.521206
2733,2733,Roberts,Texas,TX,27326,30095,Roberts,"POLYGON ((-101.086045 35.823367999999995, -101...",48393.0,3.5,...,26.3,2.45,1633278,1465992,1574266,1721726,1549434,1571632,0.036364,0.952727
3082,3082,Logan,West Virginia,WV,27332,30399,Logan,"POLYGON ((-82.196894 37.964444, -82.193913 37....",54045.0,10.1,...,10.0,2.30,248952,221847,249014,242350,222768,241678,0.169062,0.801137
2254,2254,Washington,Oregon,OR,27319,29610,Washington,"POLYGON ((-123.485441 45.447136, -123.463883 4...",41067.0,5.4,...,43.7,3.95,274302,243044,276126,324708,341167,359682,0.593956,0.327120
85,85,Nome,Alaska,AK,27283,27418,Nome,"MULTIPOLYGON (((-161.319456 64.123632, -161.31...",2180.0,11.1,...,14.6,1.25,93405,87623,85621,87647,80839,78761,0.377159,0.528870


In [5]:
gun_violence_db = pd.read_csv('/home/onyxia/work/data/df_gun_violence.csv')
gun_violence_db.sample(5)

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,...,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
42180,39639,206977,2014-10-17,Missouri,Hazelwood,8400 block of Latty Avenue,0,0,http://www.gunviolencearchive.org/incident/206977,http://www.kmov.com/news/crime/Police-still-se...,...,0,0,1,0,0,0,0,0,0,0
189170,188132,852503,2017-05-25,Maryland,Gwynn Oak,Harwall Rd and St Agnes Ln,0,1,http://www.gunviolencearchive.org/incident/852503,http://foxbaltimore.com/news/local/man-injured...,...,1,0,0,0,0,0,0,0,0,0
8161,7080,112568,2014-03-05,Colorado,Colorado Springs,North El Paso Street,0,0,http://www.gunviolencearchive.org/incident/112568,http://www.kktv.com/home/headlines/Shots-Fired...,...,0,0,1,0,0,0,0,0,0,0
63985,62534,317915,2015-04-04,Texas,Lorena,I 35,0,0,http://www.gunviolencearchive.org/incident/317915,http://kxan.com/2015/04/04/man-with-gun-halts-...,...,0,0,0,0,0,0,0,0,0,0
67430,65843,330581,2015-04-27,North Carolina,Greensboro,4409 W. Wendover Ave.,0,0,http://www.gunviolencearchive.org/incident/330581,http://myfox8.com/2015/04/27/police-looking-fo...,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [7]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']

In [8]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [9]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214 

In [10]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

Tentative de merge les deux bases

In [11]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

,id_county,geometry
0,27336,"POLYGON ((-86.92119 32.65754, -86.92035 32.658..."
1,27337,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,27338,"POLYGON ((-85.74803 31.61918, -85.74543 31.618..."
3,27339,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,27340,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."


In [12]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,...,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,geometry_y
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterfront,1,0,http://www.gunviolencearchive.org/incident/94128,http://pittsburgh.cbslocal.com/2014/01/22/u-s-...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
3,801,94504,2014-01-08,Pennsylvania,Munhall,Waterfront Shopping Complex,1,0,http://www.gunviolencearchive.org/incident/94504,http://www.post-gazette.com/local/south/2014/0...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
4,1324,95999,2014-01-10,Pennsylvania,Pittsburgh,2000 block of Fifth Avenue,0,1,http://www.gunviolencearchive.org/incident/95999,/pittsburgh.cbslocal.com/2014/01/11/man-wounde...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."


In [13]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,...,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,polygon_geometry
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterfront,1,0,http://www.gunviolencearchive.org/incident/94128,http://pittsburgh.cbslocal.com/2014/01/22/u-s-...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
3,801,94504,2014-01-08,Pennsylvania,Munhall,Waterfront Shopping Complex,1,0,http://www.gunviolencearchive.org/incident/94504,http://www.post-gazette.com/local/south/2014/0...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
4,1324,95999,2014-01-10,Pennsylvania,Pittsburgh,2000 block of Fifth Avenue,0,1,http://www.gunviolencearchive.org/incident/95999,/pittsburgh.cbslocal.com/2014/01/11/man-wounde...,...,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
